# Create journals

We need some more ways to create the journals for the batch runs. Currently, these methods are supported:
- b.from_db()
- b.from_file(filename)


## Within the Journal class

```python

    def from_file(self, file_name=None):
        """Loads a DataFrame with all the needed info about the experiment"""

        file_name = self._check_file_name(file_name)

        with open(file_name, "r") as infile:
            top_level_dict = json.load(infile)

        pages_dict = top_level_dict["info_df"]
        pages = pd.DataFrame(pages_dict)
        pages.cellpy_file_names = pages.cellpy_file_names.apply(self._fix_cellpy_paths)
        self.pages = pages
        self.file_name = file_name
        self._prm_packer(top_level_dict["metadata"])
        self.generate_folder_names()
        self.paginate()
        
```

## Within the Batch class

```python

    def create_journal(self, description=None, from_db=True):
        logging.debug("Creating a journal")
        logging.debug(f"description: {description}")
        logging.debug(f"from_db: {from_db}")
        # rename to: create_journal (combine this with function above)
        logging.info(f"name: {self.experiment.journal.name}")
        logging.info(f"project: {self.experiment.journal.project}")
        if description is not None:
            from_db = False
        if from_db:
            self.experiment.journal.from_db()
            self.experiment.journal.to_file()

        else:
            # TODO: move this into the bacth journal class
            if description is not None:
                print(f"Creating from {type(description)} is not implemented yet")

            logging.info("Creating an empty journal")
            logging.info(f"name: {self.experiment.journal.name}")
            logging.info(f"project: {self.experiment.journal.project}")

            self.experiment.journal.pages = pd.DataFrame(
                columns=[
                    "filenames",
                    "masses",
                    "total_masses",
                    "loadings",
                    "fixed",
                    "labels",
                    "cell_type",
                    "raw_file_names",
                    "cellpy_file_names",
                    "groups",
                    "sub_groups",
                ]
            )
            self.experiment.journal.pages.set_index("filenames", inplace=True)
            self.experiment.journal.generate_folder_names()
            self.experiment.journal.paginate()
            
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch
import holoviews as hv

%matplotlib inline
hv.extension('bokeh')

In [ ]:
name = "first"
project = "ocv_tests"

In [ ]:
print(" INITIALISATION OF BATCH ".center(80, "="))
b = batch.init(name, project, default_log_level="INFO")

In [ ]:
p = b.experiment.journal.create_empty_pages()

In [ ]:
filename = "20190204_FC_snx012_01_cc_03"
mass = 0.5
total_mass = 1.0
loading = 0.1
fixed = False
label = "fc_snx012_01"
cell_type = "full_cell"
raw_file_name = [Path(".") / "20190204_FC_snx012_01_cc_01.res"]
cellpy_file_name = Path(".") / "20190204_FC_snx012_01_cc_01.h5"
group = 1
sub_group = 1


p.loc[filename] = [
    mass,
    total_mass,
    loading,
    fixed,
    label,
    cell_type,
    raw_file_name,
    cellpy_file_name,
    group,
    sub_group,
]

In [ ]:
p

In [ ]:
b.pages = p

In [ ]:
b.pages

### Checking the ``create_journal`` function in ``Batch``

In [ ]:
b2 = batch.init(default_log_level="DEBUG")
b2.experiment.journal.name = "first"
b2.experiment.journal.project = "ocv_tests"

In [ ]:
# see if it finds files (str)
b2.create_journal("creating_journals_by_different_methods.ipynb")

In [ ]:
# see if it finds files (pathlib.Path)
b2.create_journal(Path("creating_journals_by_different_methods.ipynb"))

### different methods

#### dataframe

In [ ]:
filename = "20190204_FC_snx012_01_cc_03"
mass = 0.5
total_mass = 1.0
loading = 0.1
fixed = False
label = "fc_snx012_01"
cell_type = "full_cell"
raw_file_name = [Path(".") / "20190204_FC_snx012_01_cc_01.res"]
cellpy_file_name = Path(".") / "20190204_FC_snx012_01_cc_01.h5"
group = 1
sub_group = 1

d = {
    "filenames": filename,
    "masses": mass,
    "total_masses": total_mass,
    "loadings": loading,
    "fixed": fixed,
    "labels": label,
    "cell_type": cell_type,
    "raw_file_names": raw_file_name,
    "cellpy_file_names": cellpy_file_name,
    "groups": group,
    "sub_groups": sub_group,
}

d2 = {
    "filenames": [filename],
    "masses": [mass],
    "total_masses": [total_mass],
    "loadings": [loading],
    "fixed": [fixed],
    "labels": [label],
    "cell_type": [cell_type],
    "raw_file_names": [raw_file_name],
    "cellpy_file_names": [cellpy_file_name],
    "groups": [group],
    "sub_groups": [sub_group],
}

d3 = {
    "filenames": [filename, filename+"b"],
    "masses": [mass, 0.4],
    "total_masses": [total_mass, 1.0],
    "loadings": [loading, 0.2],
    "fixed": [fixed, 1],
    "labels": [label, "JPM"],
    "cell_type": [cell_type, "anode"],
    "raw_file_names": [raw_file_name, raw_file_name],
    "cellpy_file_names": [cellpy_file_name, cellpy_file_name],
    "groups": [group, 2],
    "sub_groups": [sub_group, 1],
}

d4 = {
    "filenames": [filename, filename+"b"],
    "masses": [mass],  # Different length
    "total_masses": [total_mass, 1.0],
    "loadings": [loading, 0.2],
    "fixed": [fixed, 1],
    "labels": [label, "JPM"],
    "cell_type": [cell_type, "anode"],
    "raw_file_names": [raw_file_name, raw_file_name],
    "cellpy_file_names": [cellpy_file_name, cellpy_file_name],
    "groups": [group, 2],
    "sub_groups": [sub_group, 1],
} # this should fail

d5 = {
    "filenames": [filename, filename+"b"],
    "masses": [mass, 0.2],  # Different length
    "total_masses": [total_mass, 1.0],
    "loadings": [loading, 0.2],
   # "fixed": [fixed, 1], 
    "labels": [label, "JPM"],
    "cell_type": [cell_type, "anode"],
    "raw_file_names": [raw_file_name, raw_file_name],
    "cellpy_file_names": [cellpy_file_name, cellpy_file_name],
    "groups": [group, 2],
    "sub_groups": [sub_group, 1],
}

p = pd.DataFrame(d)
p2 = pd.DataFrame(d2)
p3 = pd.DataFrame(d3)
p5 = pd.DataFrame(d5)

In [ ]:
b3 = batch.init(name, project, default_log_level="DEBUG")

In [ ]:
b3.create_journal(d)
b3.pages

In [ ]:
b3.create_journal(d2)
b3.pages

In [ ]:
b3.create_journal(d3)
b3.pages

In [ ]:
b3.create_journal(d5)
b3.pages

In [ ]:
b3.create_journal(p)
b3.pages

In [ ]:
b3.create_journal(p2)
b3.pages

In [ ]:
b3.create_journal(p3)
b3.pages

In [ ]:
b3.create_journal(p5)
b3.pages